In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [3]:
pwd

'C:\\Users\\Owner'

In [4]:
df = pd.read_csv(r'C:\\Users\\Owner\Desktop\NVDA.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2008-01-02,34.119999,34.250000,32.560001,33.009998,30.358782,12099100
1,2008-01-03,33.200001,33.860001,32.630001,32.750000,30.119659,11882700
2,2008-01-04,31.770000,31.930000,29.799999,30.000000,27.590534,18402300
3,2008-01-07,30.200001,30.450001,26.350000,26.900000,24.739513,25170000
4,2008-01-08,27.049999,29.290001,26.420000,27.469999,25.263729,27673900
...,...,...,...,...,...,...,...
3015,2019-12-23,240.520004,241.809998,238.389999,238.820007,238.497025,8876500
3016,2019-12-24,238.199997,239.309998,236.820007,238.619995,238.297287,3471600
3017,2019-12-26,238.759995,240.320007,238.000000,239.190002,238.866516,4571300
3018,2019-12-27,239.800003,240.339996,235.809998,236.869995,236.549637,6366100


In [5]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2008-01-02,34.119999,34.250000,32.560001,33.009998,30.358782,12099100
1,2008-01-03,33.200001,33.860001,32.630001,32.750000,30.119659,11882700
2,2008-01-04,31.770000,31.930000,29.799999,30.000000,27.590534,18402300
3,2008-01-07,30.200001,30.450001,26.350000,26.900000,24.739513,25170000
4,2008-01-08,27.049999,29.290001,26.420000,27.469999,25.263729,27673900


In [6]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3015,2019-12-23,240.520004,241.809998,238.389999,238.820007,238.497025,8876500
3016,2019-12-24,238.199997,239.309998,236.820007,238.619995,238.297287,3471600
3017,2019-12-26,238.759995,240.320007,238.000000,239.190002,238.866516,4571300
3018,2019-12-27,239.800003,240.339996,235.809998,236.869995,236.549637,6366100
3019,2019-12-30,235.990005,236.199997,231.059998,232.320007,232.005798,6451400


In [7]:
df = df[['Adj Close']]
df

,Adj Close
0,30.358782
1,30.119659
2,27.590534
3,24.739513
4,25.263729
...,...
3015,238.497025
3016,238.297287
3017,238.866516
3018,236.549637


In [8]:
# forecast the price for n = 30 days out into the future
forecast_out = 30
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)
print(df.tail())

       Adj Close  Prediction
3015  238.497025         NaN
3016  238.297287         NaN
3017  238.866516         NaN
3018  236.549637         NaN
3019  232.005798         NaN


C:\Users\Owner\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create the Indepedent DataSet as X
# Convert the Dataframe to a numpy array
x = np.array(df.drop(['Prediction'], 1))

In [10]:
x = x[:-forecast_out]
print(x)

[[ 30.358782]
 [ 30.119659]
 [ 27.590534]
 ...
 [209.17218 ]
 [208.134369]
 [209.351807]]


In [11]:
# Create the dependent dataset of Y values
# Convert the dataframe to a numpy array (All of the values including NaN)
y = np.array(df['Prediction'])
# Gte all of the Y values except the last 'n' rows
y = y[: -forecast_out]
print(y)

[ 20.794064  20.481365  20.177877 ... 238.866516 236.549637 232.005798]


In [12]:
# Split the Data into 80% Traning and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [13]:
# Applying Support Vector Regression
svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [14]:
# Testing Model: Score returning the coefficients of determination of prediction
# The best possible score is 1.0 as it means the predicted score and real are same
svm_confidence = svr_rbf.score(x_test, y_test)
print("SVM Confidence: ", svm_confidence)

SVM Confidence:  0.9663387508432769


In [15]:
# Create and train the Linear Regression Model 
lr = LinearRegression()
lr.fit(x_train, y_train)            # fitting the Data 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
# Testing Model: Score returning the coefficients of determination of prediction
# The best possible score is 1.0 as it means the predicted score and real are same
lr_confidence = lr.score(x_test, y_test)
print("Linear Regression Confidence: ", lr_confidence)

Linear Regression Confidence:  0.9621453540195543


In [17]:
# Set x_forecast equal to the last 30 rows of original data set from Adj.Close column
x_forecast = np.array(df.drop(['Prediction'], 1))[forecast_out:]
print(x_forecast)

[[ 20.794064]
 [ 20.481365]
 [ 20.177877]
 ...
 [238.866516]
 [236.549637]
 [232.005798]]


In [18]:
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[ 22.67064515  22.3572293   22.05304556 ... 241.24302389 238.92083349
 234.36657783]


In [19]:
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[ 19.94706631  19.73021116  19.64308944 ... 224.52593808 247.26610925
 238.31950929]


In [29]:
lr_prediction[2960:]

array([206.05953894, 214.15113633, 209.86028514, 213.05091787,
       212.05072018, 212.76087147, 223.08292645, 218.87207824,
       220.2734004 , 218.77199533, 211.27495883, 209.68348977,
       211.44511803, 210.76448326, 214.19770585, 214.19770585,
       216.00940563, 219.37255094, 226.10887551, 226.02878232,
       227.18985497, 230.33278559, 231.55393164, 237.50952449,
       241.42318496, 240.87268584, 240.67248994, 241.24302389,
       238.92083349, 234.36657783])

In [30]:
svm_prediction[2960:]

array([189.20192605, 206.73331548, 234.94794951, 195.60160769,
       205.61757957, 196.10860995, 248.0137929 , 223.55883713,
       235.10507737, 223.19442624, 223.6900725 , 232.54836447,
       219.68200922, 233.43326961, 207.38166965, 207.38166965,
       219.22349708, 226.16772152, 253.42884889, 253.05561343,
       254.00706305, 234.70436746, 235.67333202, 253.12754775,
       225.1013227 , 224.45767343, 225.09795543, 224.52593808,
       247.26610925, 238.31950929])